In [ ]:
import pandas as pd
import requests

pd.set_option('display.max_columns', None)

In [106]:
boxscore_df = pd.read_csv('EH_std_sk_stats_all_regular_no_adj_2021-10-26.csv')
boxscore_df = boxscore_df.loc[~boxscore_df['Position'].str.contains('D')]
boxscore_df = boxscore_df.loc[boxscore_df['Age'] == 18]
boxscore_df = boxscore_df.loc[boxscore_df['Draft Rd'] == 1.0]

def hf_game_log_url(df):
    return 'https://www.hockey-reference.com/players/' + df['Player'].split(' ')[1][:1].lower() + '/' + df['Player'].split(' ')[1][:5].lower() + df['Player'].split(' ')[0][:2].lower() + '01/gamelog/' + '20' + df['Season'][-2:] 
boxscore_df['HF Game Log URL'] = boxscore_df.apply(hf_game_log_url, axis=1)

url_replacement_dict = {'https://www.hockey-reference.com/players/h/hugheja01/gamelog/2020' : 'https://www.hockey-reference.com/players/h/hugheja03/gamelog/2020',
                        'https://www.hockey-reference.com/players/m/mattest01/gamelog/2013': 'https://www.hockey-reference.com/players/m/mattest02/gamelog/2013',
                        'https://www.hockey-reference.com/players/s/stützti01/gamelog/2021': 'https://www.hockey-reference.com/players/s/stuetti02/gamelog/2021',
                        'https://www.hockey-reference.com/players/h/hallta01/gamelog/2011': 'https://www.hockey-reference.com/players/h/hallta02/gamelog/2011'}

boxscore_df['HF Game Log URL'] = boxscore_df['HF Game Log URL'].replace(url_replacement_dict)

boxscore_df.sample(5)

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,G,A1,A2,Points,iSF,iFF,iCF,ixG,Sh%,FSh%,xFSh%,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,HF Game Log URL
9892,Patrik Laine,PATRIK.LAINE,8479339.0,16-17,WPG,R,R,1998-04-19,18,2016,1.0,2.0,73,1307.60,36,13,15,64,204,284,360,18.17,17.65,12.68,6.40,33,55,43,80,61,13,14,0,0,1,0,3,-3,https://www.hockey-reference.com/players/l/lai...
2880,Connor McDavid,CONNOR.MCDAVID,8478402.0,15-16,EDM,C,L,1997-01-13,18,2015,1.0,1.0,45,849.82,16,16,16,48,105,137,156,15.21,15.24,11.68,11.10,10,17,33,12,49,9,18,0,0,9,249,355,-106,https://www.hockey-reference.com/players/m/mcd...
3424,David Pastrnak,DAVID.PASTRNAK,8477956.0,14-15,BOS,R,R,1996-05-25,18,2014,1.0,25.0,46,642.77,10,8,9,27,93,135,175,8.01,10.75,7.41,5.93,11,30,25,20,56,4,11,0,0,7,5,12,-7,https://www.hockey-reference.com/players/p/pas...
8637,Mika Zibanejad,MIKA.ZIBANEJAD,8476459.0,11-12,OTT,C,R,1993-04-18,18,2011,1.0,6.0,9,116.10,0,0,1,1,12,19,25,1.40,0.00,0.00,7.37,5,1,4,15,10,1,1,0,0,0,22,28,-6,https://www.hockey-reference.com/players/z/zib...
4045,Eeli Tolvanen,EELI.TOLVANEN,8480009.0,17-18,NSH,R,L,1999-04-22,18,2017,1.0,30.0,3,36.33,0,0,0,0,3,5,6,0.48,0.00,0.00,9.60,1,1,0,2,4,0,0,0,0,0,0,0,0,https://www.hockey-reference.com/players/t/tol...


In [107]:
def lookup_first_7_games(df):
    url = df['HF Game Log URL']
    try:
        df = pd.read_html(requests.get(url).text)[0]
        df = df.iloc[:7,:]
        df.fillna(0, inplace=True)

        df[('Unnamed: 23_level_0', 'TOI')] = df[('Unnamed: 23_level_0', 'TOI')].str.split(':').str[0].astype(int) + (df[('Unnamed: 23_level_0', 'TOI')].str.split(':').str[1].astype(int)/60)

        for col_tuples in [('Scoring', 'G'), ('Scoring', 'A'), ('Scoring', 'PTS'), ('Unnamed: 20_level_0', 'S'), 
                           ('Unnamed: 22_level_0', 'SHFT'), ('Unnamed: 26_level_0', 'FOW'), ('Unnamed: 27_level_0', 'FOL')]:
            df[col_tuples] = df[col_tuples].astype(int)

        gp = len(df)
        pts = df[('Scoring', 'PTS')].sum()
        g = df[('Scoring', 'G')].sum()
        a = df[('Scoring', 'A')].sum()
        shots = df[('Unnamed: 20_level_0', 'S')].sum()
        toi = df[('Unnamed: 23_level_0', 'TOI')].sum() / len(df)
        shifts = df[('Unnamed: 22_level_0', 'SHFT')].sum() / len(df)
        fow = df[('Unnamed: 26_level_0', 'FOW')].sum()
        fol = df[('Unnamed: 27_level_0', 'FOL')].sum()

        return [gp, pts, g, a, shots, toi, shifts, fow, fol]
    except:
        display(url)
        display(df)
        
boxscore_df['First 7 Games'] = boxscore_df.apply(lookup_first_7_games, axis=1)
boxscore_df[['F7 GP', 'F7 PTS', 'F7 G', 'F7 A', 'F7 Shots', 'F7 Avg. TOI', 'F7 Avg. Shifts', 'F7 FOW', 'F7 FOL']] = pd.DataFrame(boxscore_df['First 7 Games'].tolist(), index=boxscore_df.index)

output_df = boxscore_df[['Player', 'Season', 'Team', 'Position', 'Draft Yr', 'Draft Ov', 'F7 GP', 'F7 PTS', 'F7 G', 'F7 A', 'F7 Shots', 'F7 Avg. TOI', 'F7 Avg. Shifts', 'F7 FOW', 'F7 FOL']]
output_df.to_excel('18 YO 1st Round Picks - Results from First 7 Games.xlsx')
output_df

,Player,Season,Team,Position,Draft Yr,Draft Ov,F7 GP,F7 PTS,F7 G,F7 A,F7 Shots,F7 Avg. TOI,F7 Avg. Shifts,F7 FOW,F7 FOL
209,Aleksander Barkov,13-14,FLA,C,2013,2.0,7,5,2,3,13,15.392857,20.285714,39,46
254,Alex Burmistrov,10-11,ATL,C,2010,8.0,7,0,0,0,12,14.404762,19.285714,35,48
317,Alex Galchenyuk,12-13,MTL,C,2012,3.0,7,7,1,6,12,12.780952,19.142857,30,30
382,Alex Nylander,16-17,BUF,L,2016,8.0,4,1,0,1,4,12.329167,15.500000,0,2
527,Alexis Lafrenière,20-21,NYR,L,2020,1.0,7,1,1,0,13,15.490476,20.285714,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12475,Tyler Seguin,10-11,BOS,C,2010,2.0,7,4,2,2,15,13.176190,16.571429,33,34
12517,Tyson Jost,16-17,COL,C,2016,10.0,6,1,1,0,9,15.675000,20.166667,28,42
12539,Valeri Nichushkin,13-14,DAL,R,2013,10.0,7,1,0,1,3,13.430952,18.285714,0,1
12797,William Eklund,21-22,S.J,C/L,2021,7.0,4,3,0,3,3,12.916667,18.000000,0,2
